# Naive Bayes

In deze opdracht ga je Naive Bayes gebruiken om het onderscheid tussen berichten uit twee verschillende nieuwsgroepen te leren.  
Voor informatie over de dataset, zie: http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html

In [1]:
# misc data processing imports
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# dataset
from sklearn.datasets import fetch_20newsgroups

# classifier & testing
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score

# to parse regular expressions
import re

## 1. Data laden

  * Laad de train en test data van de 20newsgroups dataset. 
    * Vraag hierbij alleen de categorieën 'comp.sys.mac.hardware' en 'comp.sys.ibm.pc.hardware' op.
    * Zorg er ook voor dat header, footers en quotes weggelaten worden.
  * Hoeveel train samples (nieuwsgroep artikelen) zijn er? En hoeveel test samples?

In [2]:
categories = ['comp.sys.mac.hardware', 'comp.sys.ibm.pc.hardware']

newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=categories,
                                      remove=('headers', 'footers', 'quotes'))
newsgroups_test  = fetch_20newsgroups(subset='test',
                                      categories=categories,
                                      remove=('headers', 'footers', 'quotes'))
print(len(newsgroups_train.data), len(newsgroups_test.data))

1168 777


In [6]:
print(newsgroups_train.data[1])

: >So, by going mailorder through Gateway, I save ~13%.  Plus, I get
: >technical support over the phone, free software package.
: >
: Have fun trying to get hold of technical support over the phone.  At least
: locally you can walk right up to the dealer and tell him what is wrong, and
: he has to fix it.

Phone support is quick and competent from many mail order firms, but not so
quick and not so competent from others (Gateway included).  But my experience
with computer retailers (which is significant) has lead to the conclusion that
sales personnel and retail-technical personnel are forbidden to actually learn
about the products they sell.  Talk about incompetent!  O.K., so a few percent
of their answers are correct, but those salesmen don't even realize how stupid
they are.   ...  .......  O.K.  ...I'll settle down now....  .... let me
catch my breath.....  ..

Fact: retail stores never provide a better value in terms of price per product.

Retail outlets are desirable, however, to

## 2. Count vectorizer

Je gaat nu zelf de data omzetten naar count vectors, waarop Naive Bayes getraind kan worden.

### 2.1 Pre-processing

  * Splits elk newsgroup artikel in de train dataset in een lijst van afzonderlijke woorden.  
  Vervang daarbij ook alle hoofdletters door kleine letters.
  * Doe bovenstaande ook voor de test dataset.

In [13]:
tok_newsgroups_train = [re.split('\W+', item.lower())[:-1] for item in newsgroups_train.data]
tok_newsgroups_test  = [re.split('\W+', item.lower())[:-1] for item in newsgroups_test.data]

  * Bouw een vocabulaire op van alle unieke woorden die in de complete **train** dataset voorkomen.

In [17]:
vocab = []
for item in tok_newsgroups_train:
    for word in item:
        vocab.append(word)
vocab = set(vocab)
# Equivalent:
# vocab = set([word for item in tok_newsgroups_train for word in item])

In [19]:
dir(newsgroups_train)
type(newsgroups_train.data)

list

In [15]:
vocab

{'',
 '699',
 'soft',
 'schipper',
 'involved',
 'demand',
 '35',
 'incidentally',
 'telepath',
 'without',
 'esdi',
 'st3283a',
 'cdrom',
 '1360x1024x16',
 'sc2jh',
 'supplier',
 'ns16550a',
 'lxt340sy',
 'venerable',
 'dipswitches',
 '14m',
 'phenomenon',
 'seconds',
 'l3',
 'press',
 'mater',
 'terms',
 'anyoneand',
 'b2',
 'mirror',
 'sam',
 'mix',
 'resovled',
 'miraculously',
 'micro',
 'freeing',
 'qualitas',
 'bultman',
 'rack',
 '6t8d',
 'seagate',
 '255',
 'sequentially',
 'cohesive',
 'meaningless',
 'off',
 'suggesiton',
 'shown',
 'rid',
 'apis',
 'cummings',
 'decompression',
 'guy',
 'hobbyists',
 'adrenaline',
 'card',
 'blanking',
 'aing',
 'trinitron',
 'various',
 'restarted',
 'sierra',
 'christopher',
 'geez',
 '20845',
 '0488',
 'goto',
 'apsi',
 '64ox480',
 'recognized',
 '621',
 'neighbor',
 'hussle',
 'setup',
 'forwarding',
 'echos',
 'nibbles',
 'ddap2ale',
 'receipts',
 '3184',
 'cuny',
 '647',
 'law',
 'j16',
 '64kb',
 'obtain',
 'emx',
 'hardcore',
 '75',


  * Tel voor elk nieuwgroep artikel hoevaak elk woord uit de vocabulaire voorkomt.  
  Het resultaat is een matrix met shape = [n\_train\_samples, n\_words\_in\_vocabulary].

In [20]:
x_train = np.zeros((len(tok_newsgroups_train), len(vocab)))

In [21]:
for art_idx, art in enumerate(tok_newsgroups_train):
    for word_idx, word in enumerate(vocab):
        x_train[art_idx, word_idx] = art.count(word)

In [23]:
x_train.shape

(1168, 11924)

In [31]:
X_train = np.array([[item.count(word) for word in vocab] for item in tok_newsgroups_train])
X_test  = np.array([[item.count(word) for word in vocab] for item in tok_newsgroups_test])

X_train.shape, X_test.shape

((1168, 11924), (777, 11924))

In [29]:
for target in newsgroups_train.target[:10]:
    print(newsgroups_train.target_names[target], target)
print(newsgroups_train.target_names)

comp.sys.ibm.pc.hardware 0
comp.sys.ibm.pc.hardware 0
comp.sys.mac.hardware 1
comp.sys.ibm.pc.hardware 0
comp.sys.mac.hardware 1
comp.sys.mac.hardware 1
comp.sys.ibm.pc.hardware 0
comp.sys.ibm.pc.hardware 0
comp.sys.mac.hardware 1
comp.sys.mac.hardware 1
['comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware']


### 2.2 Naive Bayes trainen & testen

  * Maak een Multinomial Naive Bayes classifier uit SK-Learn aan.
  * Train de classifier op de count vectorized train data en bijbehorende targets uit de training set.

In [33]:
y_train = newsgroups_train.target
y_test = newsgroups_test.target

# X_train is dus de matrix met "Term Frequencies" die je hierboven hebt gemaakt.
# y_train geeft per artikel (per rij  uit X_train) aan bij welke klasse dit artikel hoort.
nb = MultinomialNB().fit(X_train, y_train)

  * Classificeer de test data met behulp van de getrainde Naive Bayes.
  * Bereken de gemiddelde F1-score (average='macro')

In [34]:
y_pred = nb.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.81183569750932172

In [39]:
from  sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[301,  91],
       [ 55, 330]])

## 3. TF-IDF vectorizer

In bovenstaande opdracht bestond een feature vector uit het aantal keer dat elk woord voorkomt in een artikel. Alhoewel de methode aardig werkt, wordt hiermee niet meegenomen dat sommige woorden belangrijker zijn om documenten van elkaar te onderscheiden. De term frequency–inverse document frequency neemt dit wel mee.

  * Maak een TF-IDF vectorizer uit SK-Learn aan.
  * Train de vectorizer op de oorspronkelijke nieuwsgroep artikelen (dus niet de gesplitste lijst uit opdracht 2).
  * Transformeer de train en de test data naar TF-IDF vectors.

In [40]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=0.001, max_df=1.0)
X_train_tfidf = vectorizer.fit_transform(newsgroups_train.data)
X_test_tfidf  = vectorizer.transform(newsgroups_test.data)

  * Maak een Multinomial Naive Bayes classifier uit SK-Learn aan.
  * Train de classifier op de TF-IDF vectorized train data en bijbehorende targets uit de training set.

In [41]:
nb = MultinomialNB().fit(X_train_tfidf, y_train)

  * Classificeer de test data met behulp van de getrainde Naive Bayes.
  * Bereken de gemiddelde F1-score (average='macro'). Vergelijk dit met de score van de count vectorizer.

In [42]:
y_pred_tfidf = nb.predict(X_test_tfidf)
f1_score(y_pred_tfidf, y_pred, average='macro')

0.88538094472501738

  * Kun je de score nog verder verbeteren doorstop woorden te verwijderen, of door de minimale of maximale document frequency aan te passen?